In [1]:
from flask import Flask
from werkzeug.wrappers import Request, Response

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np

import os
import io
import requests


app = Flask(__name__)

@app.route('/')
def index():
    return 'Hello World'

if __name__ == "__main__":
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)


In [2]:
df = pd.read_csv('Data/auto-mpg.csv', na_values=['NA', '?'])
print(df.head(10))

    mpg  cylinders  displacement  horsepower  weight  acceleration  \
0  18.0          8         307.0       130.0    3504          12.0   
1  15.0          8         350.0       165.0    3693          11.5   
2  18.0          8         318.0       150.0    3436          11.0   
3  16.0          8         304.0       150.0    3433          12.0   
4  17.0          8         302.0       140.0    3449          10.5   
5  15.0          8         429.0       198.0    4341          10.0   
6  14.0          8         454.0       220.0    4354           9.0   
7  14.0          8         440.0       215.0    4312           8.5   
8  14.0          8         455.0       225.0    4425          10.0   
9  15.0          8         390.0       190.0    3850           8.5   

   model year  origin                   car name  
0          70       1  chevrolet chevelle malibu  
1          70       1          buick skylark 320  
2          70       1         plymouth satellite  
3          70       1    

In [ ]:
# Count the null values 
df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [ ]:
# Filling the null values will the median value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

In [ ]:
# Take only the relevent columns and convert them to a numpy array (.values)
x = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin']].values
y = df['mpg'].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True)

model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)

2021-09-27 18:15:19.832913: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2021-09-27 18:15:19.844418: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 3501000000 Hz
2021-09-27 18:15:19.846031: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fffca3ce4b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-09-27 18:15:19.846087: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-09-27 18:15:19.846433: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/1000
10/10 - 0s - loss: 123.8522 - val_loss: 69.7055
Epoch 2/1000
10/10 - 0s - loss: 58.1384 - val_loss: 39.9842
Epoch 3/1000
10/10 - 0s - loss: 42.8950 - val_loss: 34.6583
Epoch 4/1000
10/10 - 0s - loss: 39.8445 - val_loss: 37.8921
Epoch 5/1000
10/10 - 0s - loss: 41.3943 - val_loss: 40.6058
Epoch 6/1000
10/10 - 0s - loss: 40.9651 - val_loss: 34.0500
Epoch 7/1000
10/10 - 0s - loss: 38.1200 - val_loss: 32.5002
Epoch 8/1000
10/10 - 0s - loss: 37.0291 - val_loss: 32.9619
Epoch 9/1000
10/10 - 0s - loss: 36.3789 - val_loss: 32.7145
Epoch 10/1000
10/10 - 0s - loss: 37.5485 - val_loss: 33.6797
Epoch 11/1000
10/10 - 0s - loss: 36.4623 - val_loss: 30.2199
Epoch 12/1000
10/10 - 0s - loss: 34.3975 - val_loss: 30.0615
Epoch 13/1000
10/10 - 0s - loss: 35.7266 - val_loss: 29.3111
Epoch 14/1000
10/10 - 0s - loss: 34.3654 - val_loss: 29.7833
Epoch 15/1000
10/10 - 0s - loss: 33.5176 - val_loss: 29.2222
Epoch 16/1000
10/10 - 0s - loss: 34.6995 - val_loss: 57.6253
Epoch 17/1000
10/10 - 0s - loss:

In [ ]:
prediction = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(prediction, y_test))
print(f'After the training the score is {score}')

After the training the score is 3.9994123474440983


In [ ]:
if not os.getcwd == 'Models':
    os.chdir('Models')
os.getcwd()
model.save(os.path.join(os.getcwd(), 'mpg_model.h5'))

In [ ]:
cols = [x for x in df.columns if x not in ('mpg', 'car name')]

print('{')
for i, name in enumerate(cols):
    print(f'{name}: (min:{df[name].min()}, max: {df[name].max()}){"," if i<(len(cols)-1) else ""}')
print('}')

{
cylinders: (min:3, max: 8),
displacement: (min:68.0, max: 455.0),
horsepower: (min:46.0, max: 230.0),
weight: (min:1613, max: 5140),
acceleration: (min:8.0, max: 24.8),
model year: (min:70, max: 82),
origin: (min:1, max: 3)
}
